In [1]:
import sys
import os
sys.path.insert(0 ,'../granger-causality')

In [2]:
from run import main

In [3]:
class NameSpace(object):
    def __init__(self, adict):
        self.__dict__.update(adict)

In [4]:
args = {
    'r': 100,
    'b': 128,
    's': 5,
    'lr': 0.1,
    'lr_type': 'static',
    'optimiser': 'sgd',
    'dataset': 'var_p_10_T_100_lag_3_d_0_k_none',
    'model': 'lekvar', 
}

args = NameSpace(args)

In [5]:
main(args)

AttributeError: 'NameSpace' object has no attribute 'loglevel'